# Flop Board Texture Explorer

Analyze flop continuation bets across common board textures using the same sizing buckets and grouped summaries as the flop c-bet explorer.


## Texture Definitions

- **Rainbow** – three distinct suits.
- **Monotone** – all cards share the same suit.
- **Two-Tone** – exactly two suits are present (a 2-1 split).
- **Paired** – the flop contains a pair (at least two cards of the same rank).
- **Connected** – the highest and lowest ranks are at most four apart (treating A as either high or low).
- **Ace-High** – an Ace appears and it is the highest card on the board.
- **Low Boards** – all cards are Ten or lower.
- **High Boards** – at least two cards are Broadway ranks (Ace, King, Queen, Jack).

Textures are evaluated independently, so a single board can appear in multiple tables when it satisfies multiple definitions.


In [1]:
from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")

PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    Path(r"T:\Dev\ignition\drivehud\drivehud.db"),
    Path("/mnt/t/Dev/ignition/drivehud/drivehud.db"),
    PROJECT_ROOT / "drivehud" / "drivehud.db",
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = "Database not found. Checked:" + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_PATH = PROJECT_ROOT / "analysis" / "cache" / "cbet_events.json"
FORCE_RELOAD = False

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")


In [2]:

import sys
from collections import Counter

import pandas as pd
import numpy as np
from IPython.display import display
from matplotlib.colors import LinearSegmentedColormap

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from analysis.sqlite_utils import connect_readonly
from analysis.cbet_utils import (
    BASE_PRIMARY_CATEGORIES,
    DEFAULT_DRAW_FLAGS,
    load_cbet_events,
    summarize_events,
    BET_TYPES,
    RAISE_TYPES,
    CALL_TYPES,
    FOLD_TYPES,
    classify_hand,
    extract_big_blind,
    parse_cards_text,
)
from analysis.flop_board_texture_utils import derive_texture



In [3]:
# --- Configuration & helpers ---
RAW_BUCKET_BOUNDS = [
    (0.00, 0.25),
    (0.25, 0.40),
    (0.40, 0.60),
    (0.60, 0.80),
    (0.80, 1.00),
    (1.00, 1.25),
    (1.25, float("inf")),
]

def _bucket_label(low: float, high: float) -> str:
    if high == float("inf"):
        return f">={low:.2f}"
    return f"[{low:.2f}, {high:.2f})"

BUCKETS = [(low, high, _bucket_label(low, high)) for (low, high) in RAW_BUCKET_BOUNDS]
del _bucket_label

PRIMARY_GROUPS_BASE = {cat: [cat] for cat in BASE_PRIMARY_CATEGORIES}
DRAW_FLAGS = DEFAULT_DRAW_FLAGS.copy()

DEFAULT_DROP_COLUMNS = {"Range", "Made Flush", "Made Straight"}

GROUPED_PRIMARY_ORDER = [
    ("Events", None),
    ("Air", ["Air"]),
    ("Weak Pair", ["Underpair", "Bottom Pair", "Middle Pair"]),
    ("Top Pair", ["Top Pair"]),
    ("Overpair", ["Overpair"]),
    ("Two Pair", ["Two Pair"]),
    ("Trips/Set", ["Trips/Set"]),
    ("Monster", ["Straight", "Flush", "Full House", "Quads"]),
    ("Draw", ["Flush Draw", "OESD/DG"]),
]

def _gradient_cmap(base_color: str) -> LinearSegmentedColormap:
    name = f"gradient_{base_color.strip('#')}"
    return LinearSegmentedColormap.from_list(name, ['#ffffff', base_color])

def _summary_records_to_df(records, drop_columns=None):
    if not records:
        return pd.DataFrame()
    df = pd.DataFrame(records)
    drop_columns = drop_columns or set()
    for col in drop_columns:
        if col in df.columns:
            df = df.drop(columns=[col])
    df = df.set_index("Bucket").T
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

def _style_heatmap(df, title, base_color="#1f77b4", fmt="{:.1f}", special_columns=None):
    if df.empty:
        return None
    data_index = df.index.difference(["Events"])
    styled = df.style.format(fmt)
    if not data_index.empty:
        styled = styled.background_gradient(
            cmap=_gradient_cmap(base_color),
            axis=None,
            subset=pd.IndexSlice[data_index, :]
        )
    if special_columns:
        first = special_columns[0]
        if first in df.columns:
            col_idx = df.columns.get_loc(first)
            styles = [
                {
                    'selector': f'th.col_heading.level0.col{col_idx}',
                    'props': [('border-left', '2px solid #64748b')],
                },
                {
                    'selector': f'td.col{col_idx}',
                    'props': [('border-left', '2px solid #64748b')],
                },
            ]
            styled = styled.set_table_styles(styles, overwrite=False)
    return styled.set_caption(title)

def _aggregate_grouped(df, group_order):
    rows = []
    for label, members in group_order:
        if members is None:
            if "Events" in df.index:
                rows.append((label, df.loc["Events"]))
            continue
        available = [member for member in members if member in df.index]
        if not available:
            continue
        rows.append((label, df.loc[available].sum()))
    if not rows:
        return pd.DataFrame()
    data = pd.DataFrame([series for _, series in rows], index=[label for label, _ in rows])
    return data

def display_heatmap_tables(summary_records, title, *, base_color="#1f77b4", group_color="#2ca25f", special_rows=None):
    records = list(summary_records)
    special_columns = []
    if special_rows:
        records.extend(special_rows)
        special_columns = [row['Bucket'] for row in special_rows if row.get('Bucket')]
    base_df = _summary_records_to_df(records, DEFAULT_DROP_COLUMNS)
    styled = _style_heatmap(base_df, title, base_color=base_color, special_columns=special_columns)
    if styled is not None:
        display(styled)
    grouped_df = _aggregate_grouped(base_df, GROUPED_PRIMARY_ORDER)
    styled_grouped = _style_heatmap(grouped_df, f"{title} (Grouped)", base_color=group_color, special_columns=special_columns)
    if styled_grouped is not None:
        display(styled_grouped)

def _filtered_primary_groups(subset_records):
    available = {event['primary'] for event in subset_records}
    groups = {}
    for name, members in PRIMARY_GROUPS_BASE.items():
        members_in_subset = [cat for cat in members if cat in available]
        if members_in_subset:
            groups[name] = members_in_subset
    if not groups:
        for cat in sorted(available):
            groups[cat] = [cat]
    return groups

def _build_special_rows(subset_records):
    specials = []
    special_specs = [
        ("All-In", lambda event: event.get("is_all_in")),
        ("1 BB", lambda event: event.get("is_one_bb")),
    ]
    for label, predicate in special_specs:
        filtered = [event for event in subset_records if predicate(event)]
        if not filtered:
            continue
        groups = _filtered_primary_groups(filtered)
        specials.extend(
            summarize_events(
                filtered,
                [(0.0, float("inf"), label)],
                groups,
                DRAW_FLAGS,
            )
        )
    return specials


In [4]:
events = load_cbet_events(DB_PATH, cache_path=CACHE_PATH, force=FORCE_RELOAD)
print(f"Loaded {len(events)} flop c-bet events.")


Loaded 5847 flop c-bet events.


In [5]:

import json
import sqlite3
import xml.etree.ElementTree as ET
from typing import Dict, List

DONK_CACHE_PATH = PROJECT_ROOT / "analysis" / "cache" / "donk_events.json"

def load_donk_events(
    db_path: Path,
    cache_path: Path | None = None,
    force: bool = False,
) -> List[Dict]:
    if cache_path and cache_path.exists() and not force:
        with cache_path.open('r', encoding='utf-8') as fh:
            return json.load(fh)

    events: List[Dict] = []
    with connect_readonly(db_path) as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        cur.execute('SELECT HandNumber, HandHistory FROM HandHistories')
        for hand_number, hand_xml in cur:
            try:
                root = ET.fromstring(hand_xml)
            except ET.ParseError:
                continue

            big_blind = extract_big_blind(root)
            if big_blind is None or big_blind <= 0:
                continue

            pocket_cards: Dict[str, List[tuple[str, int, str]]] = {}
            for node in root.findall('.//round[@no="1"]/cards'):
                player = node.attrib.get('player')
                cards = parse_cards_text(node.text)
                if player and len(cards) == 2:
                    pocket_cards[player] = cards
            if not pocket_cards:
                continue

            total_pot = 0.0
            last_aggressor: str | None = None
            donk_logged = False
            flop_cards: List[tuple[str, int, str]] | None = None
            flop_players: set[str] = set()
            flop_actions: List[tuple[str, str]] = []
            current_event: Dict | None = None
            responders_recorded: set[str] = set()
            flop_total_players: int | None = None
            pfr_has_acted = False

            rounds = sorted(root.findall('.//round'), key=lambda r: int(r.attrib.get('no', '0')))
            for rnd in rounds:
                round_no = int(rnd.attrib.get('no', '0'))
                if round_no == 2 and flop_cards is None:
                    for card_node in rnd.findall('cards'):
                        if card_node.attrib.get('type') == 'Flop':
                            flop_cards = parse_cards_text(card_node.text)
                            break
                if round_no == 2 and flop_total_players is None:
                    players_this_round = {
                        action.attrib.get('player')
                        for action in rnd.findall('action')
                        if action.attrib.get('player')
                    }
                    flop_total_players = len(players_this_round)

                for action in rnd.findall('action'):
                    player = action.attrib.get('player')
                    if not player:
                        continue
                    act_type = action.attrib.get('type')
                    try:
                        amount = float(action.attrib.get('sum') or 0.0)
                    except ValueError:
                        amount = 0.0

                    if round_no == 1 and act_type in RAISE_TYPES and amount > 0:
                        last_aggressor = player
                    elif round_no == 2:
                        prior_actions = list(flop_actions)
                        flop_actions.append((player, act_type))
                        flop_players.add(player)

                        if last_aggressor and player == last_aggressor:
                            pfr_has_acted = True

                        if (
                            not donk_logged
                            and last_aggressor
                            and player != last_aggressor
                            and not pfr_has_acted
                            and act_type in BET_TYPES
                            and amount > 0
                        ):
                            if flop_cards and player in pocket_cards and total_pot > 0:
                                classification = classify_hand(pocket_cards[player], flop_cards)
                                event = {
                                    'hand_number': hand_number,
                                    'player': player,
                                    'ratio': round(amount / total_pot, 6),
                                    'bet_amount': amount,
                                    'bet_amount_bb': amount / big_blind if big_blind else None,
                                    'bet_action_type': act_type,
                                    'is_all_in': act_type == '7',
                                    'is_one_bb': bool(big_blind) and abs(amount - big_blind) <= max(1e-6, big_blind * 1e-4),
                                    'big_blind': big_blind,
                                    'primary': classification['primary'],
                                    'hole_cards': ' '.join(card for _, _, card in pocket_cards[player]),
                                    'flop_cards': ' '.join(card for _, _, card in flop_cards),
                                    'has_flush_draw': bool(classification['flush_draw']),
                                    'has_oesd_dg': bool(classification['oesd_dg']),
                                    'made_flush': bool(classification['made_flush']),
                                    'made_straight': bool(classification['made_straight']),
                                    'made_full_house': bool(classification['made_full']),
                                    'in_position': any(actor != player for actor, _ in prior_actions),
                                    'flop_players': flop_total_players or len(flop_players),
                                    'responses': [],
                                }
                                events.append(event)
                                current_event = event
                                responders_recorded = set()
                            donk_logged = True

                        if (
                            round_no == 2
                            and current_event is not None
                            and player != current_event['player']
                            and player not in responders_recorded
                        ):
                            response_kind: str | None = None
                            if act_type in FOLD_TYPES:
                                response_kind = 'Fold'
                            elif act_type in CALL_TYPES:
                                response_kind = 'Call'
                            elif act_type in RAISE_TYPES or act_type in BET_TYPES:
                                response_kind = 'Raise'

                            if response_kind:
                                responder_primary = None
                                responder_flush_draw = False
                                responder_oesd = False
                                responder_made_flush = False
                                responder_made_straight = False
                                responder_made_full = False

                                if flop_cards and player in pocket_cards:
                                    responder_class = classify_hand(pocket_cards[player], flop_cards)
                                    responder_primary = responder_class['primary']
                                    responder_flush_draw = bool(responder_class['flush_draw'])
                                    responder_oesd = bool(responder_class['oesd_dg'])
                                    responder_made_flush = bool(responder_class['made_flush'])
                                    responder_made_straight = bool(responder_class['made_straight'])
                                    responder_made_full = bool(responder_class['made_full'])

                                current_event['responses'].append(
                                    {
                                        'player': player,
                                        'action_type': act_type,
                                        'response': response_kind,
                                        'amount': amount,
                                        'primary': responder_primary,
                                        'has_flush_draw': responder_flush_draw,
                                        'has_oesd_dg': responder_oesd,
                                        'made_flush': responder_made_flush,
                                        'made_straight': responder_made_straight,
                                        'made_full_house': responder_made_full,
                                    }
                                )
                                responders_recorded.add(player)
                    if amount > 0:
                        total_pot += amount
                if round_no != 2:
                    current_event = None

    if cache_path:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        with cache_path.open('w', encoding='utf-8') as fh:
            json.dump(events, fh, ensure_ascii=False, indent=2)

    return events


In [6]:
from analysis.flop_texture_tables import (
    PRIMARY_DISPLAY_ORDER,
    TEXTURE_ORDER,
    COMBINED_TEXTURE_ORDER,
    prepare_texture_dataframe,
    compute_primary_texture_table,
    compute_combined_texture_table,
    compute_percent_table,
    style_heatmap_table,
    style_heatmap_percentage_table,
)


In [7]:
cbet_df = prepare_texture_dataframe(events)
primary_cbet_table = compute_primary_texture_table(cbet_df)
display(style_heatmap_table(primary_cbet_table, 'Flop C-Bet Suited/Paired Textures'))
primary_cbet_percent = compute_percent_table(primary_cbet_table)
display(style_heatmap_percentage_table(primary_cbet_percent, 'Flop C-Bet Suited/Paired Textures (% of column)'))
combined_cbet_table = compute_combined_texture_table(cbet_df)
display(style_heatmap_table(combined_cbet_table, 'Flop C-Bet Connected/High/Low Textures'))
combined_cbet_percent = compute_percent_table(combined_cbet_table)
display(style_heatmap_percentage_table(combined_cbet_percent, 'Flop C-Bet Connected/High/Low Textures (% of column)'))

donk_events = load_donk_events(DB_PATH, cache_path=DONK_CACHE_PATH, force=FORCE_RELOAD)
print(f"Loaded {len(donk_events)} flop donk-bet events.")
donk_df = prepare_texture_dataframe(donk_events)
primary_donk_table = compute_primary_texture_table(donk_df)
display(style_heatmap_table(primary_donk_table, 'Flop Donk Bet Suited/Paired Textures'))
primary_donk_percent = compute_percent_table(primary_donk_table)
display(style_heatmap_percentage_table(primary_donk_percent, 'Flop Donk Bet Suited/Paired Textures (% of column)'))
combined_donk_table = compute_combined_texture_table(donk_df)
display(style_heatmap_table(combined_donk_table, 'Flop Donk Bet Connected/High/Low Textures'))
combined_donk_percent = compute_percent_table(combined_donk_table)
display(style_heatmap_percentage_table(combined_donk_percent, 'Flop Donk Bet Connected/High/Low Textures (% of column)'))



,Paired Two-tone,Paired Rainbow,Unpaired Monotone,Unpaired Two-tone,Unpaired Rainbow,Total (count),Total (pct)
Air,218,242,46,890,691,"2,087",35.7%
Draw,47,8,93,401,90,639,10.9%
Weak Pair,0,0,40,566,389,995,17.0%
Top Pair,0,0,35,546,363,944,16.1%
Overpair,0,0,6,248,162,416,7.1%
Two Pair,186,172,7,65,53,483,8.3%
Trips/Set,29,49,7,82,49,216,3.7%
Monster,7,12,19,22,7,67,1.1%
Total (count),487,483,253,"2,820","1,804","5,847",100.0%
Total (pct),8.3%,8.3%,4.3%,48.2%,30.9%,"5,847",100.0%


,Paired Two-tone,Paired Rainbow,Unpaired Monotone,Unpaired Two-tone,Unpaired Rainbow,Total (count),Total (pct)
Air,44.8%,50.1%,18.2%,31.6%,38.3%,35.7%,35.7%
Draw,9.7%,1.7%,36.8%,14.2%,5.0%,10.9%,10.9%
Weak Pair,0.0%,0.0%,15.8%,20.1%,21.6%,17.0%,17.0%
Top Pair,0.0%,0.0%,13.8%,19.4%,20.1%,16.1%,16.1%
Overpair,0.0%,0.0%,2.4%,8.8%,9.0%,7.1%,7.1%
Two Pair,38.2%,35.6%,2.8%,2.3%,2.9%,8.3%,8.3%
Trips/Set,6.0%,10.1%,2.8%,2.9%,2.7%,3.7%,3.7%
Monster,1.4%,2.5%,7.5%,0.8%,0.4%,1.1%,1.1%
Total (count),100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%
Total (pct),8.3%,8.3%,4.3%,48.2%,30.9%,5847.0%,100.0%


,Paired,Unpaired,Connected,Disconnected,Ace-High,Low Board,High Board,Total (count),Total (pct)
Air,460,"1,627",564,"1,523",354,738,542,"5,808",34.9%
Draw,55,584,215,424,111,207,251,"1,847",11.1%
Weak Pair,0,995,175,820,294,168,372,"2,824",17.0%
Top Pair,0,944,158,786,328,138,360,"2,714",16.3%
Overpair,0,416,129,287,0,246,64,"1,142",6.9%
Two Pair,358,125,245,238,129,163,187,"1,445",8.7%
Trips/Set,78,138,96,120,60,46,111,649,3.9%
Monster,19,48,53,14,13,20,32,199,1.2%
Total (count),970,"4,877","1,635","4,212","1,289","1,726","1,919","16,628",100.0%
Total (pct),5.8%,29.3%,9.8%,25.3%,7.8%,10.4%,11.5%,"16,628",100.0%


,Paired,Unpaired,Connected,Disconnected,Ace-High,Low Board,High Board,Total (count),Total (pct)
Air,47.4%,33.4%,34.5%,36.2%,27.5%,42.8%,28.2%,34.9%,34.9%
Draw,5.7%,12.0%,13.1%,10.1%,8.6%,12.0%,13.1%,11.1%,11.1%
Weak Pair,0.0%,20.4%,10.7%,19.5%,22.8%,9.7%,19.4%,17.0%,17.0%
Top Pair,0.0%,19.4%,9.7%,18.7%,25.4%,8.0%,18.8%,16.3%,16.3%
Overpair,0.0%,8.5%,7.9%,6.8%,0.0%,14.3%,3.3%,6.9%,6.9%
Two Pair,36.9%,2.6%,15.0%,5.7%,10.0%,9.4%,9.7%,8.7%,8.7%
Trips/Set,8.0%,2.8%,5.9%,2.8%,4.7%,2.7%,5.8%,3.9%,3.9%
Monster,2.0%,1.0%,3.2%,0.3%,1.0%,1.2%,1.7%,1.2%,1.2%
Total (count),100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%
Total (pct),5.8%,29.3%,9.8%,25.3%,7.8%,10.4%,11.5%,16628.0%,100.0%


Loaded 1403 flop donk-bet events.


,Paired Two-tone,Paired Rainbow,Unpaired Monotone,Unpaired Two-tone,Unpaired Rainbow,Total (count),Total (pct)
Air,25,44,10,141,97,317,22.6%
Draw,16,1,28,128,29,202,14.4%
Weak Pair,0,0,11,193,121,325,23.2%
Top Pair,0,0,14,165,127,306,21.8%
Overpair,0,0,1,22,9,32,2.3%
Two Pair,54,46,7,34,17,158,11.3%
Trips/Set,11,14,0,10,5,40,2.9%
Monster,1,4,10,7,1,23,1.6%
Total (count),107,109,81,700,406,"1,403",100.0%
Total (pct),7.6%,7.8%,5.8%,49.9%,28.9%,"1,403",100.0%


,Paired Two-tone,Paired Rainbow,Unpaired Monotone,Unpaired Two-tone,Unpaired Rainbow,Total (count),Total (pct)
Air,23.4%,40.4%,12.3%,20.1%,23.9%,22.6%,22.6%
Draw,15.0%,0.9%,34.6%,18.3%,7.1%,14.4%,14.4%
Weak Pair,0.0%,0.0%,13.6%,27.6%,29.8%,23.2%,23.2%
Top Pair,0.0%,0.0%,17.3%,23.6%,31.3%,21.8%,21.8%
Overpair,0.0%,0.0%,1.2%,3.1%,2.2%,2.3%,2.3%
Two Pair,50.5%,42.2%,8.6%,4.9%,4.2%,11.3%,11.3%
Trips/Set,10.3%,12.8%,0.0%,1.4%,1.2%,2.9%,2.9%
Monster,0.9%,3.7%,12.3%,1.0%,0.2%,1.6%,1.6%
Total (count),100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%
Total (pct),7.6%,7.8%,5.8%,49.9%,28.9%,1403.0%,100.0%


,Paired,Unpaired,Connected,Disconnected,Ace-High,Low Board,High Board,Total (count),Total (pct)
Air,69,248,108,209,58,119,89,900,22.5%
Draw,17,185,79,123,42,82,55,583,14.6%
Weak Pair,0,325,80,245,56,110,89,905,22.6%
Top Pair,0,306,51,255,71,100,91,874,21.8%
Overpair,0,32,16,16,0,27,3,94,2.3%
Two Pair,100,58,77,81,27,66,53,462,11.5%
Trips/Set,25,15,22,18,7,15,14,116,2.9%
Monster,5,18,15,8,4,9,11,70,1.7%
Total (count),216,"1,187",448,955,265,528,405,"4,004",100.0%
Total (pct),5.4%,29.6%,11.2%,23.9%,6.6%,13.2%,10.1%,"4,004",100.0%


,Paired,Unpaired,Connected,Disconnected,Ace-High,Low Board,High Board,Total (count),Total (pct)
Air,31.9%,20.9%,24.1%,21.9%,21.9%,22.5%,22.0%,22.5%,22.5%
Draw,7.9%,15.6%,17.6%,12.9%,15.8%,15.5%,13.6%,14.6%,14.6%
Weak Pair,0.0%,27.4%,17.9%,25.7%,21.1%,20.8%,22.0%,22.6%,22.6%
Top Pair,0.0%,25.8%,11.4%,26.7%,26.8%,18.9%,22.5%,21.8%,21.8%
Overpair,0.0%,2.7%,3.6%,1.7%,0.0%,5.1%,0.7%,2.3%,2.3%
Two Pair,46.3%,4.9%,17.2%,8.5%,10.2%,12.5%,13.1%,11.5%,11.5%
Trips/Set,11.6%,1.3%,4.9%,1.9%,2.6%,2.8%,3.5%,2.9%,2.9%
Monster,2.3%,1.5%,3.3%,0.8%,1.5%,1.7%,2.7%,1.7%,1.7%
Total (count),100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%
Total (pct),5.4%,29.6%,11.2%,23.9%,6.6%,13.2%,10.1%,4004.0%,100.0%


In [8]:
primary_groups_all = _filtered_primary_groups(events)
summary_all = summarize_events(events, BUCKETS, primary_groups_all, DRAW_FLAGS)
special_all = _build_special_rows(events)
display_heatmap_tables(summary_all, "All C-Bets", special_rows=special_all)


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,410.0,2367.0,1760.0,741.0,401.0,33.0,135.0,189.0,140.0
Air,48.3,48.0,44.0,37.8,31.9,48.5,40.7,33.9,52.9
Underpair,10.7,10.1,10.5,7.6,7.5,6.1,7.4,11.1,10.0
Bottom Pair,3.2,2.5,2.7,2.8,2.5,3.0,3.0,1.6,5.7
Middle Pair,6.1,5.6,7.3,3.2,7.2,9.1,3.0,5.3,9.3
Top Pair,12.9,16.5,16.1,20.9,21.2,9.1,18.5,17.5,10.0
Overpair,4.9,4.8,6.8,13.2,14.7,15.2,13.3,14.8,1.4
Two Pair,7.6,7.3,8.8,9.4,10.2,6.1,9.6,10.1,5.0
Trips/Set,5.6,4.2,2.7,3.8,3.5,3.0,2.2,4.2,4.3
Straight,0.0,0.5,0.9,0.9,0.2,0.0,0.7,0.5,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,410.0,2367.0,1760.0,741.0,401.0,33.0,135.0,189.0,140.0
Air,48.3,48.0,44.0,37.8,31.9,48.5,40.7,33.9,52.9
Weak Pair,20.0,18.1,20.5,13.6,17.2,18.2,13.3,18.0,25.0
Top Pair,12.9,16.5,16.1,20.9,21.2,9.1,18.5,17.5,10.0
Overpair,4.9,4.8,6.8,13.2,14.7,15.2,13.3,14.8,1.4
Two Pair,7.6,7.3,8.8,9.4,10.2,6.1,9.6,10.1,5.0
Trips/Set,5.6,4.2,2.7,3.8,3.5,3.0,2.2,4.2,4.3
Monster,0.7,1.2,1.2,1.2,1.2,0.0,2.2,1.6,1.4
Draw,11.7,10.6,12.2,11.2,11.5,12.1,13.3,12.7,14.3


In [9]:
RANK_VALUES = {
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    'T': 10,
    'J': 11,
    'Q': 12,
    'K': 13,
    'A': 14,
}

BROADWAY_RANKS = {'A', 'K', 'Q', 'J'}

def parse_flop_cards(card_string: str | None):
    if not card_string:
        return None, None, None
    tokens = [token.strip().upper() for token in card_string.split() if token.strip()]
    if len(tokens) != 3:
        return None, None, None
    suits, ranks, values = [], [], []
    for token in tokens:
        suit, rank = token[0], token[1]
        if rank not in RANK_VALUES:
            return None, None, None
        suits.append(suit)
        ranks.append(rank)
        values.append(RANK_VALUES[rank])
    return suits, ranks, values

def is_connected(values):
    if not values or len(values) != 3 or any(v is None for v in values):
        return False
    sorted_vals = sorted(values)
    if sorted_vals[-1] - sorted_vals[0] <= 4:
        return True
    if 14 in sorted_vals:
        alt = sorted(1 if v == 14 else v for v in values)
        return alt[-1] - alt[0] <= 4
    return False

TEXTURE_SPECS = [
    ("Rainbow Flops", lambda suits, ranks, values: len(set(suits)) == 3),
    ("Monotone Flops", lambda suits, ranks, values: len(set(suits)) == 1),
    ("Two-Tone Flops", lambda suits, ranks, values: len(set(suits)) == 2),
    ("Paired Flops", lambda suits, ranks, values: len(set(ranks)) < 3),
    ("Connected Flops", lambda suits, ranks, values: is_connected(values)),
    ("Ace-High Flops", lambda suits, ranks, values: values is not None and values and 14 in values and max(values) == 14),
    ("Low Flops (All ≤ Ten)", lambda suits, ranks, values: values is not None and values and all(v <= 10 for v in values)),
    ("High Flops (≥2 Broadways)", lambda suits, ranks, values: ranks is not None and sum(1 for r in ranks if r in BROADWAY_RANKS) >= 2),
]


def _filter_events(predicate):
    matched = []
    for event in events:
        suits, ranks, values = parse_flop_cards(event.get('flop_cards'))
        if suits is None:
            continue
        if predicate(suits, ranks, values):
            matched.append(event)
    return matched


def display_texture_summary(title, predicate):
    subset = _filter_events(predicate)
    print(f"{title}: {len(subset)} events")
    if not subset:
        return
    primary_groups = _filtered_primary_groups(subset)
    summary = summarize_events(subset, BUCKETS, primary_groups, DRAW_FLAGS)
    specials = _build_special_rows(subset)
    display_heatmap_tables(summary, title, special_rows=specials)

for texture_title, texture_predicate in TEXTURE_SPECS:
    display_texture_summary(texture_title, texture_predicate)


Rainbow Flops: 2287 events


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,159.0,983.0,675.0,257.0,164.0,14.0,35.0,55.0,55.0
Air,46.5,48.7,43.7,40.1,24.4,42.9,40.0,29.1,56.4
Underpair,10.7,9.2,10.2,6.2,11.0,7.1,8.6,9.1,9.1
Bottom Pair,3.1,2.5,2.4,1.9,2.4,0.0,5.7,1.8,5.5
Middle Pair,6.3,5.5,6.5,1.2,8.5,14.3,2.9,1.8,7.3
Top Pair,12.6,15.2,16.3,21.0,21.3,7.1,11.4,16.4,5.5
Overpair,3.8,4.8,7.0,12.1,15.9,7.1,11.4,14.5,nan
Two Pair,8.8,8.3,10.4,12.5,12.2,14.3,14.3,18.2,7.3
Trips/Set,7.5,5.1,2.7,4.3,3.7,7.1,0.0,5.5,9.1
Straight,0.0,0.3,0.4,0.4,0.0,0.0,0.0,nan,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,159.0,983.0,675.0,257.0,164.0,14.0,35.0,55.0,55.0
Air,46.5,48.7,43.7,40.1,24.4,42.9,40.0,29.1,56.4
Weak Pair,20.1,17.2,19.1,9.3,22.0,21.4,17.1,12.7,21.8
Top Pair,12.6,15.2,16.3,21.0,21.3,7.1,11.4,16.4,5.5
Overpair,3.8,4.8,7.0,12.1,15.9,7.1,11.4,14.5,0.0
Two Pair,8.8,8.3,10.4,12.5,12.2,14.3,14.3,18.2,7.3
Trips/Set,7.5,5.1,2.7,4.3,3.7,7.1,0.0,5.5,9.1
Monster,0.6,0.7,0.9,0.8,0.6,0.0,5.7,3.6,0.0
Draw,6.9,3.6,5.2,3.1,4.9,7.1,0.0,1.8,9.1


Monotone Flops: 253 events


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,26.0,87.0,76.0,32.0,24.0,1.0,7.0,14.0,13.0
Air,53.8,39.1,28.9,28.1,33.3,100.0,42.9,35.7,53.8
Underpair,15.4,17.2,19.7,6.2,8.3,0.0,28.6,35.7,nan
Bottom Pair,0.0,1.1,3.9,3.1,0.0,0.0,0.0,nan,nan
Middle Pair,3.8,10.3,9.2,0.0,4.2,0.0,0.0,nan,7.7
Top Pair,15.4,14.9,13.2,40.6,16.7,0.0,14.3,21.4,23.1
Overpair,0.0,5.7,10.5,9.4,16.7,0.0,0.0,nan,nan
Two Pair,0.0,1.1,6.6,0.0,4.2,0.0,0.0,nan,nan
Trips/Set,3.8,2.3,1.3,6.2,0.0,0.0,14.3,7.1,nan
Straight,0.0,0.0,3.9,3.1,4.2,0.0,0.0,nan,nan


...

Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,103.0,540.0,600.0,275.0,141.0,11.0,56.0,74.0,44.0
Air,72.8,57.4,55.2,38.9,31.9,63.6,51.8,48.6,77.3
Underpair,3.9,3.3,4.0,4.7,6.4,0.0,1.8,4.1,4.5
Bottom Pair,1.9,4.1,2.8,2.5,3.5,0.0,0.0,nan,2.3
Middle Pair,2.9,3.5,5.0,1.5,7.1,0.0,0.0,1.4,4.5
Top Pair,1.0,8.0,7.7,14.2,9.9,0.0,8.9,6.8,2.3
Overpair,6.8,10.7,11.5,24.0,27.7,18.2,26.8,27.0,2.3
Two Pair,5.8,8.3,10.3,10.5,10.6,18.2,7.1,8.1,2.3
Trips/Set,4.9,3.3,1.8,2.5,2.1,0.0,3.6,4.1,4.5
Straight,0.0,0.7,0.8,1.1,0.0,0.0,0.0,nan,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,103.0,540.0,600.0,275.0,141.0,11.0,56.0,74.0,44.0
Air,72.8,57.4,55.2,38.9,31.9,63.6,51.8,48.6,77.3
Weak Pair,8.7,10.9,11.8,8.7,17.0,0.0,1.8,5.4,11.4
Top Pair,1.0,8.0,7.7,14.2,9.9,0.0,8.9,6.8,2.3
Overpair,6.8,10.7,11.5,24.0,27.7,18.2,26.8,27.0,2.3
Two Pair,5.8,8.3,10.3,10.5,10.6,18.2,7.1,8.1,2.3
Trips/Set,4.9,3.3,1.8,2.5,2.1,0.0,3.6,4.1,4.5
Monster,0.0,1.3,1.7,1.1,0.7,0.0,0.0,0.0,0.0
Draw,14.6,12.8,13.7,10.9,11.3,27.3,8.9,12.2,22.7


High Flops (≥2 Broadways): 1275 events


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,105.0,598.0,321.0,143.0,79.0,6.0,23.0,35.0,37.0
Air,39.0,40.8,33.6,32.9,26.6,33.3,30.4,17.1,40.5
Underpair,12.4,10.7,11.2,6.3,5.1,16.7,17.4,20.0,8.1
Bottom Pair,4.8,2.5,3.7,3.5,3.8,16.7,0.0,2.9,8.1
Middle Pair,9.5,7.5,9.7,5.6,7.6,0.0,8.7,8.6,18.9
Top Pair,14.3,18.1,19.3,26.6,31.6,0.0,30.4,25.7,8.1
Overpair,1.9,1.2,2.5,4.2,5.1,16.7,0.0,2.9,nan
Two Pair,8.6,10.4,12.1,12.6,11.4,0.0,4.3,8.6,8.1
Trips/Set,8.6,7.0,5.9,7.0,7.6,16.7,0.0,8.6,5.4
Straight,0.0,0.5,1.6,1.4,1.3,0.0,0.0,nan,nan


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,105.0,598.0,321.0,143.0,79.0,6.0,23.0,35.0,37.0
Air,39.0,40.8,33.6,32.9,26.6,33.3,30.4,17.1,40.5
Weak Pair,26.7,20.7,24.6,15.4,16.5,33.3,26.1,31.4,35.1
Top Pair,14.3,18.1,19.3,26.6,31.6,0.0,30.4,25.7,8.1
Overpair,1.9,1.2,2.5,4.2,5.1,16.7,0.0,2.9,0.0
Two Pair,8.6,10.4,12.1,12.6,11.4,0.0,4.3,8.6,8.1
Trips/Set,8.6,7.0,5.9,7.0,7.6,16.7,0.0,8.6,5.4
Monster,1.0,1.8,1.9,1.4,1.3,0.0,8.7,5.7,2.7
Draw,17.1,13.0,16.8,11.9,10.1,0.0,21.7,20.0,10.8
